In [1]:
# lib
import sys

sys.path.append("..")

# set cwd one up
import os

os.chdir("..")
path = "data/hes/Hes1_example.csv"

os.getcwd()

'/Users/tsd/Python/GPCell'

In [2]:
from gpcell import utils

import numpy as np

In [ ]:
par1 = np.array([300, 1, 0.07, 0.07, 1, 1, 0], dtype=np.float64)
par2 = np.array([100, 3, 0.03, 0.03, 1, 1, 18], dtype=np.float64)
n_cells = 500  # total replicates used in the simulation
t_final_vals = [1500, 600]  # 25/10 hours
noise_vals = [np.sqrt(x) for x in [0.1, 0.5]]

# X, Y = utils.get_time_series(par1, par2, t_final_vals[0], noise_vals[0], n_cells)
# print(len(X), len(Y))
# print(X[0].shape, Y[0].shape)

In [4]:
# from gpcell.backend import _simulate_replicate_mod9

Nvec = [20]
out_times = np.arange(5000, 5000 + t_final_vals[0] + 1, 30)

N = Nvec[0]
mstart = 1 * N
pstart = 30 * N

# for i in range(10):
#     _, _ = _simulate_replicate_mod9(N, par2, mstart, pstart, out_times)

In [5]:
path = "data/results/"
# generate data
for noise in noise_vals:
    X, Y = utils.get_time_series(par1, par2, t_final_vals[0], noise, n_cells)

    # save data
    utils.save_sim(
        X,
        Y,
        path + f"noise_{round(noise**2, 2)}_time_{t_final_vals[0]}_rep_{n_cells}.csv",
    )

X, Y = utils.get_time_series(par1, par2, t_final_vals[1], noise_vals[1], n_cells)
utils.save_sim(
    X,
    Y,
    path
    + f"noise_{round(noise_vals[1] ** 2, 2)}_time_{t_final_vals[1]}_rep_{n_cells}.csv",
)

gillespie_timing_mod9_nodelay: Replicate 20 finished.
gillespie_timing_mod9_nodelay: Replicate 30 finished.
gillespie_timing_mod9_nodelay: Replicate 10 finished.
gillespie_timing_mod9_nodelay: Replicate 40 finished.
gillespie_timing_mod9_nodelay: Replicate 50 finished.
gillespie_timing_mod9_nodelay: Replicate 60 finished.
gillespie_timing_mod9_nodelay: Replicate 70 finished.
gillespie_timing_mod9_nodelay: Replicate 80 finished.
gillespie_timing_mod9_nodelay: Replicate 90 finished.
gillespie_timing_mod9_nodelay: Replicate 100 finished.
gillespie_timing_mod9_nodelay: Replicate 110 finished.
gillespie_timing_mod9_nodelay: Replicate 120 finished.
gillespie_timing_mod9_nodelay: Replicate 130 finished.
gillespie_timing_mod9_nodelay: Replicate 140 finished.
gillespie_timing_mod9_nodelay: Replicate 150 finished.
gillespie_timing_mod9_nodelay: Replicate 160 finished.
gillespie_timing_mod9_nodelay: Replicate 170 finished.
gillespie_timing_mod9_nodelay: Replicate 180 finished.
gillespie_timing_mo